In [ ]:
from sympy import S, expand_trig, trigsimp

from pandas import DataFrame, set_option, Series

from numpwd.qchannels.spin import get_spin_matrix_element, dict_to_data
from numpwd.qchannels.lsj import project_op
from numpwd.integrate.analytic import SPHERICAL_BASE_SUBS, ANGLE_BASE_SUBS, integrate

set_option("max_colwidth", None)

In [ ]:
sig1_k1 = S("sigma11 * l11 + sigma12  * l12 + sigma13 * l13")
sig2_k2 = S("sigma21 * l21 + sigma22  * l22 + sigma23 * l23")

kernel = sig1_k1 * sig2_k2
kernel = kernel.subs(
    {"l11": "+p_i1 - p_o1", "l12": "+p_i2 - p_o2", "l13": "+p_i3 - p_o3 + q/2"}
)
kernel = kernel.subs(
    {"l21": "-p_i1 + p_o1", "l22": "-p_i2 + p_o2", "l23": "-p_i3 + p_o3 + q/2"}
)
kernel

In [ ]:
decomposition = get_spin_matrix_element(kernel)
decomp_df = (
    DataFrame(decomposition).set_index(["s_o", "ms_o", "s_i", "ms_i"]).sort_index()
)
decomp_df

In [ ]:
from numpwd.qchannels.cg import get_cg, Number

In [ ]:
df = decomp_df.copy()

In [ ]:
res = project_op(decomposition, "s_o", "s_i")
ddf = DataFrame(dict_to_data(res, columns=["s_o", "s_i", "sigma", "m_sigma"]))
ddf.head()

In [ ]:
ddf = (
    DataFrame(dict_to_data(res, columns=["s_o", "s_i", "sigma", "m_sigma"]))
)
ddf["res"] = ddf.apply(
    lambda el: integrate(
        expand_trig(
            el["val"].subs(SPHERICAL_BASE_SUBS).subs(ANGLE_BASE_SUBS)
            * S(f"exp(I * {el['m_sigma']} * (Phi - phi /2))")
        ).expand()
    ),
    axis=1,
)

ddf = ddf.set_index(["s_o", "s_i", "sigma", "m_sigma"]).sort_index()
ddf.head()

In [ ]:
alpha = S("p_i**2 + p_o**2 - q**2/4")
beta = S("exp(I * phi) * p_i * sqrt(1 - x_i**2) - p_o * sqrt(1 - x_o**2)")
delta = S("p_i * x_i - p_o * x_o")
gamma = S("p_i * p_o * (x_i * x_o  + cos(phi)* sqrt(1-x_i**2) * sqrt(1-x_o**2))")
q = S("q")
pi = S("pi")

In [ ]:
(ddf.loc[(0,0,0,0), "res"] - 2 * pi * (alpha - 2*gamma)).simplify()

In [ ]:
(ddf.loc[(0,1,1,-1), "res"] + S("sqrt(6)") * pi * beta * q).simplify()